In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from netCDF4 import Dataset
import pandas as pd
# import properscoring as ps
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import warnings
import xarray as xr
from matplotlib import cm
from utils.evaluation import find_landfalling_tcs,tc_region,create_xarray,get_storm_coords
from utils.metrics import calculate_crps
from global_land_mask import globe
from scipy.interpolate import griddata
from utils.data import load_tc_data,accumulated_rain
from utils.plot import make_cmap,plot_accumulated
import cartopy.crs as ccrs
# from utils.metrics import calculate_fid
import xesmf as xe

import glob
warnings.filterwarnings("ignore")

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [ ]:
# load datasets
print('loading datasets', flush=True)
real,inputs,pred,meta = load_tc_data(set='validation',results='ke_tracks')
_,_,_,_,_,_,pred_2,meta_2 = load_tc_data(set='validation',results='test')
miroc_fp = '/user/home/al18709/work/gan_predictions_20/miroc_pred-opt_no_rain_test_run_1.npy'
miroc_corrected_fp = '/user/home/al18709/work/gan_predictions_20/miroc_corrected_pred-opt_no_rain_test_run_1.npy'
miroc = np.load(miroc_fp)
filepath = '/user/home/al18709/work/ibtracks/miroc6_ssp585_tracks.csv'
miroc_meta = pd.read_csv(filepath)
miroc_corrected = np.load(miroc_corrected_fp)
print('datasets loaded!', flush=True)

real_1D,inputs_1D,pred_1D,meta_1D = load_tc_data(set='validation',results='ke_tracks')

In [ ]:
print(real.shape)
print(pred.shape)
print(meta.shape)
print(meta_2.shape)
print(real_1D.shape)
print(meta_1D.shape)
print(pred_2.shape)

In [ ]:
valid_sids = list(dict.fromkeys(meta['sid']))
# get valid sids
# assign sid variable to list of sid indices correspoinnding to storm timesteps
for sid in valid_sids:
	indices = meta.sid[meta.sid == sid].index.tolist()
	print(indices)
	exec('sid_%s = indices' % sid)

new_sids = list(dict.fromkeys(meta_2['sid']))
# get valid sids
# assign sid variable to list of sid indices correspoinnding to storm timesteps
for sid in new_sids:
	indices = meta_2.sid[meta_2.sid == sid].index.tolist()
	print(indices)
	exec('sid_2_%s = indices' % sid)

In [ ]:
# get tracks
tracks = pd.read_csv('/user/work/al18709/ibtracks/tc_files.csv')

In [ ]:
print(sid_2011233N15301)

In [ ]:
# Irene 2011233N15301 validation NA
accumulated_ds_1,accumulated_ds_pred_1 = accumulated_rain(sid_2011233N15301,meta,real[:,:,:,0],pred[:,:,:,0],inputs)
accumulated_1D,accumulated_ds_pred_1D = accumulated_rain(sid_2011233N15301,meta_1D,real_1D[:,:,:,0],pred_1D[:,:,:,0],inputs)

meta_irene = pd.read_csv('/user/work/al18709/tc_data_mswep_extend_flipped/meta_2011233N15301.csv')
irene_real = np.load('/user/home/al18709/work/gan_predictions_20/storm_real-opt_2011233N15301.npy')[:,:,:,0]
irene_pred = np.load('/user/home/al18709/work/gan_predictions_20/storm_pred-opt_2011233N15301.npy')[:,:,:,0]
irene_inputs = np.load('/user/home/al18709/work/gan_predictions_20/storm_input-opt_2011233N15301.npy')[:,:,:,0]
sid_irene_2011233N15301 = np.arange(len(irene_real[:,0,0]))
accumulated_ds_2,accumulated_ds_pred_2 = accumulated_rain(sid_irene_2011233N15301,meta_irene,irene_real,irene_pred,irene_inputs)
storm_1 = '2011233N15301'

# storm_1 = '2011233N15301'
# get tracks
centre_lats_1 = tracks['lat'].loc[tracks['sid']==storm_1].reset_index(drop=True)
centre_lons_1 = tracks['lon'].loc[tracks['sid']==storm_1].reset_index(drop=True)
intensity_1 = tracks['sshs'].loc[tracks['sid']==storm_1].reset_index(drop=True)

In [ ]:
print(accumulated_ds_2.precipitation.shape)
print(accumulated_ds_pred_2.precipitation.shape)
print(accumulated_ds_1.precipitation.shape)
print(accumulated_ds_pred_1.precipitation.shape)
print(accumulated_1D.precipitation.shape)
print(accumulated_ds_pred_1D.precipitation.shape)
print(meta_irene)
print(meta.loc[sid_2011233N15301])


In [ ]:
print(real_1D[sid_2011233N15301[0],:,:,0].shape)
print(real_1D[sid_2011233N15301[0],:,:,0].shape)

print(sid_2011233N15301)
print(sid_2_2011233N15301)

In [ ]:
levels = [10, 15, 20, 25, 30, 40, 50,75, 100,120,130, 150,180,200,250,300,350,400,450,500]
sns.set_style("white")
sns.set_context("notebook")
# define cmap
precip_cmap,precip_norm = make_cmap()

# gridspec = {'width_ratios': [1,1,0.2,1,1, 0.2]}
# fig, axes = plt.subplots(3, 6, figsize=(32, 20), sharey=False,subplot_kw={'projection': ccrs.PlateCarree()}, gridspec_kw=gridspec)
fig, axes = plt.subplots(3, 10, figsize=(100, 25), sharey=False,subplot_kw={'projection': ccrs.PlateCarree()}) # gridspec_kw=gridspec)

axes[0,0].axis('off')
axes[0,1].axis('off')
axes[1,0].axis('off')
axes[1,1].axis('off')

lons = accumulated_ds_1['lat'].values[0:100]
lats = accumulated_ds_1['lon'].values[0:100]
vmax=50

axes[0,0] = plot_accumulated(real_1D[sid_2011233N15301[20],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,0],mask_oceans=True,title='title')
axes[0,1] = plot_accumulated(real_1D[sid_2011233N15301[21],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,1],mask_oceans=True,title='title')
axes[0,2] = plot_accumulated(real_1D[sid_2011233N15301[22],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,2],mask_oceans=True,title='title')
axes[0,3] = plot_accumulated(real_1D[sid_2011233N15301[23],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,3],mask_oceans=True,title='title')
axes[0,4] = plot_accumulated(real_1D[sid_2011233N15301[24],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,4],mask_oceans=True,title='title')
axes[0,5] = plot_accumulated(real_1D[sid_2011233N15301[25],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,5],mask_oceans=True,title='title')
axes[0,6] = plot_accumulated(real_1D[sid_2011233N15301[26],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,6],mask_oceans=True,title='title')
axes[0,7] = plot_accumulated(real_1D[sid_2011233N15301[27],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,7],mask_oceans=True,title='title')
axes[0,8] = plot_accumulated(real_1D[sid_2011233N15301[28],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,8],mask_oceans=True,title='title')
axes[0,9] = plot_accumulated(real_1D[sid_2011233N15301[29],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[0,9],mask_oceans=True,title='title')
axes[1,0] = plot_accumulated(pred_1D[sid_2011233N15301[20],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,0],mask_oceans=True,title='title')
axes[1,1] = plot_accumulated(pred_1D[sid_2011233N15301[21],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,1],mask_oceans=True,title='title')
axes[1,2] = plot_accumulated(pred_1D[sid_2011233N15301[22],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,2],mask_oceans=True,title='title')
axes[1,3] = plot_accumulated(pred_1D[sid_2011233N15301[23],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,3],mask_oceans=True,title='title')
axes[1,4] = plot_accumulated(pred_1D[sid_2011233N15301[24],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,4],mask_oceans=True,title='title')
axes[1,5] = plot_accumulated(pred_1D[sid_2011233N15301[25],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,5],mask_oceans=True,title='title')
axes[1,6] = plot_accumulated(pred_1D[sid_2011233N15301[26],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,6],mask_oceans=True,title='title')
axes[1,7] = plot_accumulated(pred_1D[sid_2011233N15301[27],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,7],mask_oceans=True,title='title')
axes[1,8] = plot_accumulated(pred_1D[sid_2011233N15301[28],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,8],mask_oceans=True,title='title')
axes[1,9] = plot_accumulated(pred_1D[sid_2011233N15301[29],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[1,9],mask_oceans=True,title='title')

axes[2,0] = plot_accumulated(pred_2[sid_2_2011233N15301[20],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,0],mask_oceans=True,title='title')
axes[2,1] = plot_accumulated(pred_2[sid_2_2011233N15301[21],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,1],mask_oceans=True,title='title')
axes[2,2] = plot_accumulated(pred_2[sid_2_2011233N15301[22],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,2],mask_oceans=True,title='title')
axes[2,3] = plot_accumulated(pred_2[sid_2_2011233N15301[23],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,3],mask_oceans=True,title='title')
axes[2,4] = plot_accumulated(pred_2[sid_2_2011233N15301[24],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,4],mask_oceans=True,title='title')
axes[2,5] = plot_accumulated(pred_2[sid_2_2011233N15301[25],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,5],mask_oceans=True,title='title')
axes[2,6] = plot_accumulated(pred_2[sid_2_2011233N15301[26],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,6],mask_oceans=True,title='title')
axes[2,7] = plot_accumulated(pred_2[sid_2_2011233N15301[27],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,7],mask_oceans=True,title='title')
axes[2,8] = plot_accumulated(pred_2[sid_2_2011233N15301[28],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,8],mask_oceans=True,title='title')
axes[2,9] = plot_accumulated(pred_2[sid_2_2011233N15301[29],:,:,0],lats,lons,
							vmin=0,vmax=vmax,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=None,
							centre_lons=None,intensity=intensity_1,ax=axes[2,9],mask_oceans=True,title='title')


# Irene 2011233N15301 validation NA
xlim_1 = (-80,-74)
ylim_1 = (33,39)


# axes[0,0].set_xlim(xlim_1)
# axes[0,0].set_ylim(ylim_1)
# axes[0,1].set_xlim(xlim_1)
# axes[0,1].set_ylim(ylim_1)
# axes[1,0].set_xlim(xlim_1)
# axes[1,0].set_ylim(ylim_1)
# axes[1,1].set_xlim(xlim_1)
# axes[1,1].set_ylim(ylim_1)

# axes[1].set_xlim(xlim_1)
# axes[1].set_ylim(ylim_1)
plt.savefig('hurricane_irene4.png')

In [ ]:
levels = [10, 15, 20, 25, 30, 40, 50,75, 100,120,130, 150,180,200,250,300,350,400,450,500]
sns.set_style("white")
sns.set_context("notebook")
# define cmap
precip_cmap,precip_norm = make_cmap()

# gridspec = {'width_ratios': [1,1,0.2,1,1, 0.2]}
# fig, axes = plt.subplots(3, 6, figsize=(32, 20), sharey=False,subplot_kw={'projection': ccrs.PlateCarree()}, gridspec_kw=gridspec)
fig, axes = plt.subplots(2, 2, figsize=(10, 10), sharey=False,subplot_kw={'projection': ccrs.PlateCarree()}) # gridspec_kw=gridspec)

axes[0,0].axis('off')
axes[0,1].axis('off')
axes[1,0].axis('off')
axes[1,1].axis('off')

axes[0,0] = plot_accumulated(accumulated_ds_1['precipitation'],accumulated_ds_1['lat'].values,accumulated_ds_1['lon'].values,
							vmin=0,vmax=375,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=centre_lats_1,
							centre_lons=centre_lons_1,intensity=intensity_1,ax=axes[0,0],mask_oceans=True,title='MSWEP')
axes[0,1] = plot_accumulated(accumulated_1D['precipitation'],accumulated_ds_1['lat'].values,accumulated_ds_1['lon'].values,
							vmin=0,vmax=375,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=centre_lats_1,
							centre_lons=centre_lons_1,intensity=intensity_1,ax=axes[0,1],mask_oceans=True,title='')
axes[1,0] = plot_accumulated(accumulated_ds_pred_2['precipitation'],accumulated_ds_1['lat'].values,accumulated_ds_1['lon'].values,
							vmin=0,vmax=375,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=centre_lats_1,
							centre_lons=centre_lons_1,intensity=intensity_1,ax=axes[1,0],mask_oceans=True,title='Original WGAN')
axes[1,1] = plot_accumulated(accumulated_ds_pred_1D['precipitation'],accumulated_ds_1['lat'].values,accumulated_ds_1['lon'].values,
							vmin=0,vmax=375,cmap=precip_cmap,levels = levels,plot='ax',centre_lats=centre_lats_1,
							centre_lons=centre_lons_1,intensity=intensity_1,ax=axes[1,1],mask_oceans=True,title='1D WGAN')


# Irene 2011233N15301 validation NA
xlim_1 = (-80,-74)
ylim_1 = (33,39)


axes[0,0].set_xlim(xlim_1)
axes[0,0].set_ylim(ylim_1)
axes[0,1].set_xlim(xlim_1)
axes[0,1].set_ylim(ylim_1)
axes[1,0].set_xlim(xlim_1)
axes[1,0].set_ylim(ylim_1)
axes[1,1].set_xlim(xlim_1)
axes[1,1].set_ylim(ylim_1)

# axes[1].set_xlim(xlim_1)
# axes[1].set_ylim(ylim_1)
plt.savefig('hurricane_irene_axccumulated_4.png')